In [10]:
import os
import sys


curr_dir = os.getcwd()
parent_dir = os.path.dirname(curr_dir)
sys.path.append(parent_dir)


import psutil
from utils import check_and_install
    
# List of packages to check, with any special names or setup functions
packages = [
    "pandas",
    "numpy",
    "matplotlib",
    "seaborn",
    "scikit-learn",
    "scipy",
    "tqdm",
    "pandarallel",
    "thinc",
    "srsly",
    "fastai"
]

# Loop through each package, importing and installing if needed
for package in packages:
    check_and_install(package)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



from pandarallel import pandarallel

real_cpu_count = psutil.cpu_count(logical=False)
pandarallel.initialize(progress_bar=True)


scikit-learn not found, installing...
scikit-learn has been installed
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
DatasetRaces = pd.read_csv("../dataset/races_filled.csv")
DatasetCyclists = pd.read_csv("../dataset/cyclists_filled.csv")

MergedDataset = pd.merge(DatasetCyclists, DatasetRaces.drop(columns=['name']), left_on='_url', right_on='cyclist', suffixes=('_cyclist', '_race')).drop(columns=['_url_cyclist']) 


print(MergedDataset.duplicated().sum())
print(MergedDataset.isnull().sum())

MergedDataset.sample(3)


0
name                      0
birth_year                0
weight                    0
height                    0
nationality               0
expected_url              0
_url_race                 0
points                    0
uci_points                0
length                    0
climb_total               0
profile                   0
startlist_quality         0
date                      0
position                  0
cyclist                   0
cyclist_age               0
is_tarmac                 0
is_cobbled                0
is_gravel                 0
cyclist_team         158681
delta                     0
race_year                 0
dtype: int64


,name,birth_year,weight,height,nationality,expected_url,_url_race,points,uci_points,length,...,date,position,cyclist,cyclist_age,is_tarmac,is_cobbled,is_gravel,cyclist_team,delta,race_year
476806,Andrey Grivko,1983.0,70.000000,181.000000,Ukraine,andrey--grivko,itzulia-basque-country/2008/stage-1,0.0,0.0,137000.0,...,2008-04-07 03:39:50,41,andrei-grivko,25.0,True,False,False,NaN,34.0,2008
269250,Simon Geschke,1986.0,64.000000,171.000000,Germany,simon--geschke,giro-d-italia/2015/stage-9,35.0,4.0,224000.0,...,2015-05-17 05:50:54,3,simon-geschke,29.0,True,False,False,belkin-pro-cycling-team-2014,23.0,2015
72979,Gert Jakobs,1964.0,68.890299,186.110345,Netherlands,gert--jakobs,tour-de-france/1988/stage-19,0.0,0.0,188000.0,...,1988-07-21 05:31:43,143,gert-jakobs,24.0,True,False,False,NaN,1029.0,1988


### Outliers analysis

Since each run of the algorithms might produce different parameters and different outliers, we will use the outliers generated by the main notebook.

In [9]:
outliers_lof = pd.read_csv('../dataset/outliers_lof.csv')
outliers_iso_for = pd.read_csv('../dataset/outliers_iso_for.csv')
outliers_oc_svm_gpu = pd.read_csv('../dataset/outliers_oc_svm.csv')

# normal_lof = MergedDataset[~MergedDataset['_url_race'].isin(outliers_lof['_url'])]
outliers_lof.sample(3)

,nationality,pca_race_points,physical_score,race_difficulty,combined_score_per_km,consistency_score,race_performance_score
3079,61,-0.195077,0.657600,-2.575559,19.367620,4.158911,0.966010
5739,52,0.410889,1.171305,0.443537,-0.263474,-0.334492,0.036322
4850,69,0.485388,-0.629432,1.134599,-0.283370,-0.133105,-1.433834
